In [1]:
import typing

import pandas as pd

import config
import src
import requests
import tqdm
import json
import numpy as np

from sklearn.metrics import cohen_kappa_score, classification_report
import krippendorff
import yaml


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\sstolwi\Git

In [53]:
CFG = config.Config()

In [3]:
dataset: pd.DataFrame = pd.read_csv('data/publicsphere/full_data.csv')
dataset

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,dislikeCount_video,likeCount_video,date_difference,commentCount_video,replyCount_comment,topic,subscribers,HATELIST_FOCUSED_DUMMY,Time_comment_year,Time_video_year
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2017,2017.0
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,195.0,3817.0,743.0,1748.0,NaN,economy,3630000.0,0,2019,2019.0
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,8/19/2021 14:50:13,8/19/2021 14:54:28,62.194.51.29,1,6,1152219467579100000,10000695,0,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3858,8/19/2021 15:10:27,8/19/2021 15:12:21,62.194.51.29,1,6,1085362296472430000,10007008,1,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3859,10/6/2021 16:08:39,10/6/2021 16:10:42,213.127.113.113,1,6,UghFY3QJ6nmT_ngCoAEC.7-H0Z7--wxd8goqpaPs-bl,20000102,0,3,1,...,2820.0,12475.0,3803.0,4785.0,NaN,east,6740000.0,0,2018,2010.0
3860,10/15/2021 18:30:04,10/15/2021 18:35:40,213.127.109.191,1,6,UgyWabsmmnq3zam4DgZ4AaABAg,20000418,2,3,1,...,118.0,31761.0,1531.0,2206.0,0.0,east,6800000.0,0,2018,2015.0


In [42]:
MODEL: str = 'llama3.1:70b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
CFG.prompt_classify_files.keys()

dict_keys(['diversity_disagreement', 'diversity_disagreement_edit', 'diversity_ideological_direction', 'diversity_positiondum', 'incivility_allcaps', 'incivility_attackreputation', 'incivility_combine', 'incivility_namecalling', 'incivility_namecalling_edit', 'incivility_question_intelligence', 'incivility_sarcasm', 'incivility_simple', 'incivility_vulgarity', 'interactivity_acknowledgement', 'intolerance_discrimination', 'intolerance_rights', 'intolerance_violence', 'political_ideology', 'political_ideology_ext', 'political_ideology_GER', 'political_ideology_GER2', 'political_ideology_GER3', 'political_ideology_US', 'political_negativity', 'rationality_background_info', 'rationality_background_info_edit', 'rationality_combine', 'rationality_external_evidence', 'rationality_external_evidence_edit', 'rationality_reasoning', 'rationality_reasoning_edit', 'rationality_simple', 'rationality_topic_relevance'])

In [5]:
#change options to low temperature (0,1) and compare result:
options_low = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low)

In [56]:
#inspect GLLM reasoning to ensure valid processing of task and correct task interpretation:
            
predictions: typing.Dict[str, np.ndarray] = {}
for index, row in tqdm.tqdm(dataset["commentText"][:5].items()):
    try: 
        output = np.array(
            requests.post(
                'https://inf.cl.uni-trier.de/',
                json={
                    'model': MODEL,
                    'prompt': f"Does this comment acknowledge a previously posted user-comment or claim of another discussant? \nInstruction: By referring back to another comment, one recognizes the existence of a previous comment. Acknowledgment can be neutral or by explicitly endorsing another discussant's comment (e.g. 'I absolutely agree with what you're saying'). Keywords such as 'Yes; Yep; Exactly; I totally agree' may signal acknowledgement of a comment of another discussant. Acknowledgement can also be explicitly disagreeing with another discussant's comment or claim (e.g. '@RoniBox No, you're not right. Climate change is not real!'). \nA comment coded as YES for Acknowledgement is also likely to be coded as YES for Interaction. \nExample 1: '@tirtha simanta yeah but now someone deleted all of it' \nExample 2: 'Nope, that is not true' (is 1 under isReply) \nException 1: Code as NO if there is NOT a @username (or +1username, or username) NOR it is NOT a reply, because in most cases that means the comment is not referring to a commenter involved in the discussion. Whether this comment is a reply or not is indicated by 'isReply' ('1' means it is a reply) \nException 2: Code as NO if the comment is simply criticizing or cursing at another discussant (e.g. @Roompie Think properly, you fool), without explicitly saying that the writer (dis)agrees with another discussant. \\n\\n First explain your reasoning, then respond with the predicted class (0 or 1) of the request.\\n\\nText: {row}"                        
                    }).json()['response'])                  
        
                    
    except json.JSONDecodeError:
        print("invalid json response, skipping to next batch")
        output = None
        
    predictions[index] = output

0it [00:00, ?it/s]

5it [00:24,  4.94s/it]


In [57]:
predictions


{0: array(' This comment does not acknowledge a previously posted user-comment or claim of another discussant. The text simply says "sad" which doesn\'t refer to any specific comment or user. Therefore, I would predict that this comment is coded as NO for Acknowledgement and also NO for Interaction since it doesn\'t interact with any other comments.',
       dtype='<U338'),
 1: array(' This comment does not appear to acknowledge a previously posted user-comment or claim of another discussant, as there is no explicit reference to or endorsement of another user\'s comment. The comment contains a playful tone, addressing "a box of rocks" and claiming to represent the "coalition for mineral rights," but it does not engage with any specific points raised by other users. Therefore, I would predict that this request should be coded as 0 (NO) for Acknowledgement.\n\nThat being said, without more context, it is possible that this comment is a reply to a specific user or thread of comments that 

apparently, it Llama3.1 expects the input text to mention the isReply-variable (which it does not), which might bias the results of the coding
mixtral however seems to ignore this part of the instruction
overall the reasoning provided does not indicate serious interpretation problems of the model with regards to the instructions

In [6]:
#make a list of prompts to be classified:
pubsphereprompts = ['diversity_disagreement', 'diversity_ideological_direction', 'diversity_positiondum', 'incivility_allcaps', 'incivility_attackreputation', 'incivility_namecalling', 'incivility_question_intelligence', 'incivility_sarcasm', 'incivility_vulgarity', 'interactivity_acknowledgement', 'intolerance_discrimination', 'intolerance_rights', 'intolerance_violence', 'political_ideology_US', 'rationality_background_info', 'rationality_external_evidence', 'rationality_reasoning', 'rationality_topic_relevance']

In [40]:
pubspherepromptsrun14 = ['incivility_namecalling', 'incivility_namecalling_edit']   
predictions14: typing.Dict[str, np.ndarray] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["commentText"], model=MODEL, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label in pubspherepromptsrun14
}

classifying incivility_namecalling_edit: 100%|██████████| 3862/3862 [28:54<00:00,  2.23it/s]   


In [13]:
#load the dataset:
dataset_w_pred_2 = pd.read_json(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.json')
dataset_w_pred_anon = pd.read_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [42]:
#join to the dataset:   
for _, preds in predictions14.items():
    print(preds.value_counts())
    print("-" * 42)
    dataset_w_pred_2 = dataset_w_pred_2.join(preds, rsuffix='_updated')
    dataset_w_pred_anon = dataset_w_pred_anon.join(preds, rsuffix='_updated')

incivility_namecalling
No     2670
Yes    1189
Name: count, dtype: int64
------------------------------------------
incivility_namecalling_edit
No     2486
Yes    1371
Name: count, dtype: int64
------------------------------------------


In [8]:
dataset_w_pred_2.columns.to_list()

['StartDate',
 'RecordedDate',
 'IPAddress',
 'Finished',
 'Coder',
 'ID',
 'Mark_ID',
 'Genre',
 'topiccode',
 'Platform',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'TopicRelevance',
 'Reasoning',
 'BackgroundInfo',
 'ExternalEvidence',
 'ExternalEvidence_1_TEXT',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'INTERACTIVITY_DUMMY',
 'RATIONALITY_DUMMY',
 'HAS_OPINION_DUMMY',
 'LIBERAL_NEUTRAL_CONSERVATIVE',
 'LIBERAL_DUMMY',
 'CONSERVATIVE_DUMMY',
 'NAMECALLING_DUMMY',
 'VULGAR_DUMMY',
 'NAMECALLING_VULGAR_DUMMY',
 'INCIVILITY_ORDINAL',
 'INCIVILITY_DUMMY',
 'INTOLERANCE_DUMMY',
 'filter_$',
 'IMPOLITENESS_DUMMY',
 'showName',
 'genre',
 'Time_comment',
 'likeCount_comment',
 'entities',
 'place',
 'retweet_count',
 'platform',
 'retweeted',
 '

In [10]:
#need to recode all the llama generated variables to 0,1:
dataset_w_pred_anon.loc[:, 'diversity_positiondum_dum'] = dataset_w_pred_anon.loc[:, 'diversity_positiondum'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'diversity_disagreement_dum'] = dataset_w_pred_anon.loc[:, 'diversity_disagreement'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'diversity_position_dum'] = dataset_w_pred_anon.loc[:, 'diversity_position'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_allcaps_dum'] = dataset_w_pred_anon.loc[:, 'incivility_allcaps'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_attackreputation_dum'] = dataset_w_pred_anon.loc[:, 'incivility_attackreputation'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_question_intelligence_dum'] = dataset_w_pred_anon.loc[:, 'incivility_question_intelligence'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_sarcasm_dum'] = dataset_w_pred_anon.loc[:, 'incivility_sarcasm'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_namecalling_dum'] = dataset_w_pred_anon.loc[:, 'incivility_namecalling'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_vulgarity_dum'] = dataset_w_pred_anon.loc[:, 'incivility_vulgarity'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'diversity_ideologicaldirection_dum'] = dataset_w_pred_anon.loc[:, 'diversity_ideologicaldirection'].map({"Left/Liberal/Democratic": 0, "Absent":1, "Neutral":1, "Unclear which direction":1 , "Right/Conservative/Republican":2}).fillna(0).astype(int)

dataset_w_pred_2.loc[:, 'diversity_disagreement_dum'] = dataset_w_pred_2.loc[:, 'diversity_disagreement'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_2.loc[:, 'diversity_position_dum'] = dataset_w_pred_2.loc[:, 'diversity_position'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_2.loc[:, 'incivility_allcaps_dum'] = dataset_w_pred_2.loc[:, 'incivility_allcaps'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_2.loc[:, 'incivility_attackreputation_dum'] = dataset_w_pred_2.loc[:, 'incivility_attackreputation'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_2.loc[:, 'incivility_question_intelligence_dum'] = dataset_w_pred_2.loc[:, 'incivility_question_intelligence'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_2.loc[:, 'incivility_sarcasm_dum'] = dataset_w_pred_2.loc[:, 'incivility_sarcasm'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_2.loc[:, 'incivility_namecalling_dum'] = dataset_w_pred_2.loc[:, 'incivility_namecalling'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_2.loc[:, 'incivility_vulgarity_dum'] = dataset_w_pred_2.loc[:, 'incivility_vulgarity'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

dataset_w_pred_2.loc[:, 'diversity_ideologicaldirection_dum'] = dataset_w_pred_2.loc[:, 'diversity_ideologicaldirection'].map({"Left/Liberal/Democratic": 0, "Absent":1, "Neutral":1, "Unclear which direction":1 , "Right/Conservative/Republican":2}).fillna(0).astype(int)

#need to combine incivility and rationality items into seperate variables:

In [9]:
dataset_w_pred_anon.head()

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,incivility_prompt_dum,hate_list_prompt_dum,rationality_background_info,rationality_external_evidence,rationality_external_evidence_edit,rationality_background_info_edit,interactivity_acknowledgement,rationality_reasoning_edit,incivility_namecalling_updated,incivility_namecalling_edit
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,0,0,No,No,No,No,No,No,No,No
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,1,1,No,No,No,No,No,Yes,Yes,Yes
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,1,0,Yes,Yes,Yes,Yes,Yes,Yes,No,No
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,0,0,No,No,Yes,Yes,No,Yes,No,No
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,1,0,No,No,No,No,No,No,No,No


In [95]:
dataset_w_pred_2.to_json(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_2.to_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.parquet')
dataset_w_pred_anon.to_json('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_anon.to_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [53]:
#load data
dataset_w_pred_anon = pd.read_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [65]:
dataset_w_pred_anon.columns.to_list()

['StartDate',
 'RecordedDate',
 'IPAddress',
 'Finished',
 'Coder',
 'ID',
 'Mark_ID',
 'Genre',
 'topiccode',
 'Platform',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'TopicRelevance',
 'Reasoning',
 'BackgroundInfo',
 'ExternalEvidence',
 'ExternalEvidence_1_TEXT',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'INTERACTIVITY_DUMMY',
 'RATIONALITY_DUMMY',
 'HAS_OPINION_DUMMY',
 'LIBERAL_NEUTRAL_CONSERVATIVE',
 'LIBERAL_DUMMY',
 'CONSERVATIVE_DUMMY',
 'NAMECALLING_DUMMY',
 'VULGAR_DUMMY',
 'NAMECALLING_VULGAR_DUMMY',
 'INCIVILITY_ORDINAL',
 'INCIVILITY_DUMMY',
 'INTOLERANCE_DUMMY',
 'filter_$',
 'IMPOLITENESS_DUMMY',
 'showName',
 'genre',
 'Time_comment',
 'likeCount_comment',
 'entities',
 'place',
 'retweet_count',
 'platform',
 'retweeted',
 '

In [56]:
#rename old variabels by adding '_old'  ['interactivity_acknowledgement',  'rationality_external_evidence', 'rationality_topic_relevance', 'political_negativity', 'rationality_background_info', 'rationality_reasoning', 'political_ideology_US', 'rationality_reasoning_dum', 'rationality_background_info_dum', 'rationality_external_evidence_dum', 'rationality_prompt_dum', 'political_conservative_US', 'political_liberal_US', 'political_opinion_US', 'diversity_positiondum']:
#dataset_w_pred_anon.rename(columns={'interactivity_acknowledgement': 'interactivity_acknowledgement_old', 'rationality_external_evidence': 'rationality_external_evidence_old', 'rationality_topic_relevance': 'rationality_topic_relevance_old', 
#                                    'political_negativity': 'political_negativity_old', 'rationality_background_info': 'rationality_background_info_old', 'rationality_reasoning': 'rationality_reasoning_old', 'political_ideology_US': 'political_ideology_US_old', 
#                                    'rationality_reasoning_dum': 'rationality_reasoning_dum_old', 'rationality_background_info_dum': 'rationality_background_info_dum_old', 'rationality_external_evidence_dum': 'rationality_external_evidence_dum_old', 
#                                    'rationality_prompt_dum': 'rationality_prompt_dum_old', 'political_conservative_US': 'political_conservative_US_old', 'political_liberal_US' : 'political_liberal_US_old', 'political_opinion_US': 'political_opinion_US_old', 
#                                    'diversity_positiondum': 'diversity_positiondum_old', 'diversity_positiondum_dum': 'diversity_positiondum_dum_old' }, inplace=True)
#rename 'incivility_namecalling_dum' and 'incivility_namecalling' to indicate they have missings:
#dataset_w_pred_anon.rename(columns={'incivility_namecalling_dum': 'incivility_namecalling_dum_NA', 'incivility_namecalling': 'incivility_namecalling_NA'}, inplace=True)                                    
#rename 'incivility_namecalling_dum_low2' as standard:
#dataset_w_pred_anon.rename(columns={'incivility_namecalling_low2': 'incivility_namecalling'}, inplace=True) 
#rename variables to indicate they are a duplicate run of the same variable:
#dataset_w_pred_anon.rename(columns={'rationality_background_info': 'rationality_background_info_run2'}, inplace=True)
#dataset_w_pred_anon.rename(columns={'rationality_external_evidence': 'rationality_external_evidence_run2'}, inplace=True)
#rename variables to indicate they are an updated prompt version of the same variable:
#dataset_w_pred_anon.rename(columns={'interactivity_acknowledgement': 'interactivity_acknowledgement_updated'}, inplace=True)


###Note: the _edit-variables are the newest, followed by the _updated and not suffixed ones, and the _old ones are the oldest.


In [57]:
#make dummy variables for the items that are not yet dummified:
dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_dum'] = dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_low'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_external_evidence_dum'] = dataset_w_pred_anon.loc[:, 'rationality_external_evidence_low'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_topic_relevance_dum'] = dataset_w_pred_anon.loc[:, 'rationality_topic_relevance_low'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_background_info_low_dum'] = dataset_w_pred_anon.loc[:, 'rationality_background_info_low'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_reasoning_dum'] = dataset_w_pred_anon.loc[:, 'rationality_reasoning_low'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'intolerance_discrimination_dum'] = dataset_w_pred_anon.loc[:, 'intolerance_discrimination'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'intolerance_rights_dum'] = dataset_w_pred_anon.loc[:, 'intolerance_rights'].map({"Yes": 1, "No":0}).fillna(0).astype(int)   
dataset_w_pred_anon.loc[:, 'intolerance_violence_dum'] = dataset_w_pred_anon.loc[:, 'intolerance_violence'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_namecalling_dum'] = dataset_w_pred_anon.loc[:, 'incivility_namecalling'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'political_conservative_US_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_low'].map({"conservative": 1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'political_liberal_US_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_low'].map({"liberal": 1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'political_opinion_US_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_low'].map({"liberal": 1, 'conservative': 1}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'diversity_conservative_dum'] = dataset_w_pred_anon.loc[:, 'diversity_ideologicaldirection'].map({"Left/Liberal/Democratic": 0, "Absent":0, "Neutral":0, "Unclear which direction":0 , "Right/Conservative/Republican":1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'diversity_liberal_dum'] = dataset_w_pred_anon.loc[:, 'diversity_ideologicaldirection'].map({"Left/Liberal/Democratic": 1, "Absent":0, "Neutral":0, "Unclear which direction":0 , "Right/Conservative/Republican":0}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_updated_dum'] = dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_updated'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_background_info_updated_dum'] = dataset_w_pred_anon.loc[:, 'rationality_background_info_updated'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_external_evidence_updated_dum'] = dataset_w_pred_anon.loc[:, 'rationality_external_evidence_updated'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_namecalling_updated_dum'] = dataset_w_pred_anon.loc[:, 'incivility_namecalling_updated'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'incivility_namecalling_edit_dum'] = dataset_w_pred_anon.loc[:, 'incivility_namecalling_edit'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_reasoning_edit_dum'] = dataset_w_pred_anon.loc[:, 'rationality_reasoning_edit'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_background_info_run2_dum'] = dataset_w_pred_anon.loc[:, 'rationality_background_info_run2'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_external_evidence_run2_dum'] = dataset_w_pred_anon.loc[:, 'rationality_external_evidence_run2'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'rationality_external_evidence_edit_dum'] = dataset_w_pred_anon.loc[:, 'rationality_external_evidence_edit'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_background_info_edit_dum'] = dataset_w_pred_anon.loc[:, 'rationality_background_info_edit'].map({"Yes": 1, "No":0}).fillna(0).astype(int)



In [58]:
#create compound-concept variables with the latest version of the prompt:
dataset_w_pred_anon.loc[:, 'rationality_prompt_dum'] = dataset_w_pred_anon.loc[:, ['rationality_reasoning_edit_dum','rationality_background_info_edit_dum','rationality_external_evidence_edit_dum']].max(axis=1)
dataset_w_pred_anon.loc[:, 'incivility_prompt_dum'] = dataset_w_pred_anon.loc[:, ['incivility_namecalling_edit_dum', 'incivility_vulgarity_dum', 'incivility_attackreputation_dum', 
                'incivility_question_intelligence_dum', 'incivility_allcaps_dum', 'incivility_sarcasm_dum',
              'intolerance_rights_dum', 'intolerance_discrimination_dum', 'intolerance_violence_dum']].max(axis=1)
dataset_w_pred_anon.loc[:, 'hate_list_prompt_dum'] = dataset_w_pred_anon.loc[:, ['incivility_namecalling_edit_dum', 'incivility_vulgarity_dum', 'intolerance_rights_dum', 'intolerance_discrimination_dum', 'intolerance_violence_dum']].max(axis=1) 

In [14]:
dataset_w_pred_anon.loc[:, ['rationality_prompt_dum', 'rationality_reasoning_dum','rationality_background_info_low_dum','rationality_external_evidence_dum', 'rationality_external_evidence_low','rationality_background_info_low','rationality_reasoning_low' ]].head(10)

,rationality_prompt_dum,rationality_reasoning_dum,rationality_background_info_low_dum,rationality_external_evidence_dum,rationality_external_evidence_low,rationality_background_info_low,rationality_reasoning_low
0,0,0,0,0,No,No,No
1,1,1,0,0,No,No,Yes
2,1,1,1,1,Yes,Yes,Yes
3,1,1,0,0,No,No,Yes
4,1,1,0,0,No,No,Yes
5,0,0,0,0,None,No,No
6,0,0,0,0,None,No,No
7,0,0,0,0,None,No,No
8,1,1,0,0,No,No,Yes
9,1,1,0,0,No,No,Yes


In [59]:
#count NA's:
dataset_w_pred_anon.loc[:, ['interactivity_acknowledgement_low', 'rationality_external_evidence_low','rationality_background_info_low','rationality_reasoning_low', 'political_ideology_US_low', "rationality_topic_relevance_low", 
                            "political_ideology_US_low", 'incivility_namecalling', 'incivility_namecalling_NA', 'incivility_vulgarity', 'incivility_attackreputation', 
                            'incivility_question_intelligence', 'incivility_allcaps', 'incivility_sarcasm', 'intolerance_rights', 'intolerance_discrimination', 'intolerance_violence', 
                            'diversity_ideologicaldirection', 'diversity_position', 'diversity_disagreement', 'rationality_background_info_updated', 'rationality_external_evidence_updated',
                            'interactivity_acknowledgement_updated', 'incivility_namecalling_updated',
                            'incivility_namecalling_edit','rationality_reasoning_edit', 'rationality_background_info_run2','rationality_external_evidence_run2', 'rationality_external_evidence_edit', 
                            'rationality_background_info_edit']].isna().sum()

interactivity_acknowledgement_low           2
rationality_external_evidence_low         910
rationality_background_info_low            90
rationality_reasoning_low                 102
political_ideology_US_low                   0
rationality_topic_relevance_low             2
political_ideology_US_low                   0
incivility_namecalling                    156
incivility_namecalling_NA                 174
incivility_vulgarity                       16
incivility_attackreputation                 9
incivility_question_intelligence            3
incivility_allcaps                          5
incivility_sarcasm                          6
intolerance_rights                          3
intolerance_discrimination                  1
intolerance_violence                        3
diversity_ideologicaldirection              5
diversity_position                          3
diversity_disagreement                      4
rationality_background_info_updated       151
rationality_external_evidence_upda

In [ ]:
#Note many missing values...especially in external_evidence.
#but edited prompts fixed the issue, so we can use the edited prompts for the analysis.

In [33]:
#check run rerun accuracy:
print(pd.crosstab(dataset_w_pred_anon['rationality_background_info_updated'], dataset_w_pred_anon['rationality_background_info_run2'], margins=True, dropna=False))
print("---")
print(pd.crosstab(dataset_w_pred_anon['rationality_external_evidence_updated'], dataset_w_pred_anon['rationality_external_evidence_run2'], margins=True, dropna=False))

#compare to runs with and without seed/low temperature:
print("---")
print("effect of temperature:")
print(pd.crosstab(dataset_w_pred_anon['rationality_background_info_old'], dataset_w_pred_anon['rationality_background_info_low'], margins=True, dropna=False))
print(pd.crosstab(dataset_w_pred_anon['rationality_external_evidence_old'], dataset_w_pred_anon['rationality_external_evidence_low'], margins=True, dropna=False))

rationality_background_info_run2       No  Yes  NaN     All
rationality_background_info_updated                        
No                                   2690   10   21  2721.0
Yes                                    11  979    0   990.0
NaN                                    30    0  121     NaN
All                                  2731  989    0  3862.0
---
rationality_external_evidence_run2       No  Yes   NaN     All
rationality_external_evidence_updated                         
No                                     1615   11    85  1711.0
Yes                                       9  788     0   797.0
NaN                                      86    2  1266     NaN
All                                    1710  801     0  3862.0
---
effect of temperature:
rationality_background_info_low    No   Yes  NaN     All
rationality_background_info_old                         
No                               2728   343   87  3158.0
Yes                                33   667    0   700.0
NaN

much less difference in run-rerun within seed and low temperature, than in high-low temperature between seeds
Still setting the seed and use low temperature, did not produce fully identical results...
We do observe a strong increase in NAs for these variables with the change in seed an temperature, unexpectedly these were far few for the high temperature-version, perhaps a lucky random seed on that run?


In [84]:
#what is the effect of reformulating the prompt:
print(pd.crosstab(dataset_w_pred_anon['rationality_background_info_updated'], dataset_w_pred_anon['rationality_background_info_edit'], margins=True, dropna=False))
print("---")
print(pd.crosstab(dataset_w_pred_anon['rationality_external_evidence_updated'], dataset_w_pred_anon['rationality_external_evidence_edit'], margins=True, dropna=False))
print("---")
print(pd.crosstab(dataset_w_pred_anon['incivility_namecalling_updated'], dataset_w_pred_anon['incivility_namecalling_edit'], margins=True, dropna=False))
print("---")
print(pd.crosstab(dataset_w_pred_anon['incivility_namecalling'], dataset_w_pred_anon['incivility_namecalling_edit'], margins=True, dropna=False))
print("---")
print(pd.crosstab(dataset_w_pred_anon['rationality_reasoning_low'], dataset_w_pred_anon['rationality_reasoning_edit'], margins=True, dropna=False))



rationality_background_info_edit       No  Yes  NaN     All
rationality_background_info_updated                        
No                                   2640   81    0  2721.0
Yes                                    98  892    0   990.0
NaN                                   147    0    4     NaN
All                                  2885  973    0  3862.0
---
rationality_external_evidence_edit       No  Yes  NaN     All
rationality_external_evidence_updated                        
No                                     1582  129    0  1711.0
Yes                                      56  740    1   797.0
NaN                                    1337   16    1     NaN
All                                    2975  885    0  3862.0
---
incivility_namecalling_edit       No   Yes  NaN     All
incivility_namecalling_updated                         
No                              2462   206    2  2670.0
Yes                               23  1164    2  1189.0
NaN                                1

#not only effect on reduction of NAs but also quite some recoding to switch categories from yes -> no and vice versa, not as much as changing seed and temperature, but more than a simple rerun,
#also nearly all NAs were now coded as 'NO' which is equivalent to the legacy handling of these values, so these do not change the results of the classifier performance statistics, improvements are largely due to switching labels

In [86]:
#does this also hold for diversity_disagreement?
pubspherepromptsrun16 = ['diversity_disagreement_edit']   
predictions16: typing.Dict[str, np.ndarray] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["commentText"], model=MODEL, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label in pubspherepromptsrun16
}

classifying diversity_disagreement_edit: 100%|██████████| 3862/3862 [28:09<00:00,  2.29it/s]  


In [92]:
#join to the dataset:   
for _, preds in predictions16.items():
    print(preds.value_counts())
    print("-" * 42)
    dataset_w_pred_2 = dataset_w_pred_2.join(preds, rsuffix='_edit')
    dataset_w_pred_anon = dataset_w_pred_anon.join(preds, rsuffix='_edit')

diversity_disagreement_edit
Yes    2068
No     1791
Name: count, dtype: int64
------------------------------------------


In [94]:
dataset_w_pred_anon.loc[:, 'diversity_disagreement_edit_dum'] = dataset_w_pred_2.loc[:, 'diversity_disagreement_edit'].map({"Yes": 1, "No":0}).fillna(0).astype(int)


In [93]:
#compare results:
print(pd.crosstab(dataset_w_pred_anon['diversity_disagreement'], dataset_w_pred_anon['diversity_disagreement_edit'], margins=True, dropna=False))

diversity_disagreement_edit    No   Yes  NaN     All
diversity_disagreement                              
No                           1741   474    1  2216.0
Yes                            49  1593    0  1642.0
NaN                             1     1    2     NaN
All                          1791  2068    0  3862.0


In [98]:
#compare results to manual coding:
print(pd.crosstab(dataset_w_pred_anon['disagreement'], dataset_w_pred_anon['diversity_disagreement_edit'], margins=True, dropna=False))

diversity_disagreement_edit    No   Yes  NaN   All
disagreement                                      
0                            1780  1896    3  3679
1                              11   172    0   183
All                          1791  2068    0  3862


It appears the model finds vastly more disagreement, probably since it misses the context in the prompt to determine whether a post is an answer to a question, or real disagreement (since the thread is not provided in the prompt)

In [120]:
dataset_w_pred_anon.columns.to_list()

['StartDate',
 'RecordedDate',
 'IPAddress',
 'Finished',
 'Coder',
 'ID',
 'Mark_ID',
 'Genre',
 'topiccode',
 'Platform',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'TopicRelevance',
 'Reasoning',
 'BackgroundInfo',
 'ExternalEvidence',
 'ExternalEvidence_1_TEXT',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'INTERACTIVITY_DUMMY',
 'RATIONALITY_DUMMY',
 'HAS_OPINION_DUMMY',
 'LIBERAL_NEUTRAL_CONSERVATIVE',
 'LIBERAL_DUMMY',
 'CONSERVATIVE_DUMMY',
 'NAMECALLING_DUMMY',
 'VULGAR_DUMMY',
 'NAMECALLING_VULGAR_DUMMY',
 'INCIVILITY_ORDINAL',
 'INCIVILITY_DUMMY',
 'INTOLERANCE_DUMMY',
 'filter_$',
 'IMPOLITENESS_DUMMY',
 'showName',
 'genre',
 'Time_comment',
 'likeCount_comment',
 'entities',
 'place',
 'retweet_count',
 'platform',
 'retweeted',
 '

In [117]:
#evaluete respective performance:
llm_human_column_pairs = [
    ("disagreement", "diversity_disagreement_edit_dum"),
    ("disagreement", "diversity_disagreement_dum")
]
for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")

---
              precision    recall  f1-score   support

           0       0.99      0.48      0.65      3679
           1       0.08      0.94      0.15       183

    accuracy                           0.51      3862
   macro avg       0.54      0.71      0.40      3862
weighted avg       0.95      0.51      0.63      3862

cohen_kappa_score(disagreement): 0.07202694798722953
krippendorf(disagreement): -0.1954620245925982
percent_agreement(disagreement): 50.62%
---
              precision    recall  f1-score   support

           0       1.00      0.60      0.75      3679
           1       0.11      0.95      0.19       183

    accuracy                           0.62      3862
   macro avg       0.55      0.78      0.47      3862
weighted avg       0.95      0.62      0.72      3862

cohen_kappa_score(disagreement): 0.11524484045577765
krippendorf(disagreement): -0.059559293773962185
percent_agreement(disagreement): 61.76%


The results of the edited prompt are even worse than the original prompt

perhaps this is because diversity_disagreement— only applies if answer to this option was "Yes":       __Does this comment refer to a societally or __politically relevant issue (i.e. on-topic)?
So maybe we should only look at those comments for results


In [ ]:
print(pd.crosstab(dataset_w_pred_anon['TopicRelevance'], dataset_w_pred_anon['disagreement'], margins=True, dropna=False))

disagreement       0    1   All
TopicRelevance                 
0               1498   53  1551
1               2181  130  2311
All             3679  183  3862


no, coding doesn't appear to be restricted to topicrelevant comments...

In [110]:
#evaluate performance against manual coding of rationality_prompt_dum incivility_prompt_dum hate_list_prompt_dum interactivity_acknowledgement_dum, political_conservative_US_dum, political_liberal_US_dum, political_opinion_US_dum:
llm_human_column_pairs = [
    ("LIBERAL_DUMMY", "political_liberal_US_dum"),
    ("CONSERVATIVE_DUMMY", "political_conservative_US_dum"),
    ("HAS_OPINION_DUMMY", "political_opinion_US_dum"),
    ("LIBERAL_DUMMY", "diversity_liberal_dum"),
    ("CONSERVATIVE_DUMMY", "diversity_conservative__dum"),
    ("HAS_OPINION_DUMMY", "diversity_position_dum"),
    ("RATIONALITY_DUMMY", "rationality_prompt_dum"),
    ("TopicRelevance", "rationality_topic_relevance_dum"),
    ("INTERACTIVITY_DUMMY", "interactivity_acknowledgement_updated_dum"),
    ("INCIVILITY_DUMMY", "incivility_prompt_dum"),
    ("HATELIST_FOCUSED_DUMMY", "hate_list_prompt_dum")
]

In [116]:
#these are the new results of the edited prompts:

print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.94      0.85      0.89      3073
           1       0.56      0.77      0.65       789

    accuracy                           0.83      3862
   macro avg       0.75      0.81      0.77      3862
weighted avg       0.86      0.83      0.84      3862

cohen_kappa_score(LIBERAL_DUMMY): 0.5444043080556233
krippendorf(LIBERAL_DUMMY): 0.5408921571554941
percent_agreement(LIBERAL_DUMMY): 83.14%
---
              precision    recall  f1-score   support

           0       0.95      0.90      0.92      3271
           1       0.57      0.74      0.65       591

    accuracy                           0.88      3862
   macro avg       0.76      0.82      0.79      3862
weighted avg       0.89      0.88      0.88      3862

cohen_kappa_score(CONSERVATIVE_DUMMY): 0.5725715490094205
krippendorf(CONSERVATIVE_DUMMY): 0

In [128]:
pd.crosstab(dataset_w_pred_anon.Ideologicaldirection, dataset_w_pred_anon.diversity_ideologicaldirection, margins=True, dropna=False)

diversity_ideologicaldirection,Absent,Left/Liberal/Democratic,Neutral,Right/Conservative/Republican,Unclear which direction,NaN,All
Ideologicaldirection,,,,,,,
0,58,473,37,196,25,0,789
1,1290,230,190,450,72,2,2234
2,3,11,14,20,0,1,49
3,30,48,26,83,12,0,199
4,44,29,25,483,8,2,591
All,1425,791,292,1232,117,0,3862


In [137]:
#recode the diversity_ideologicaldirection variable:
dataset_w_pred_anon.loc[:, 'diversity_ideologicaldirection_recode'] = dataset_w_pred_anon.loc[:, 'diversity_ideologicaldirection'].map({"Absent":"1", "Left/Liberal/Democratic":"0", "Neutral":"2", "Unclear wich direction":"3", "Right/Conservative/Republican":4}).fillna(1).astype(int)

In [138]:
pd.crosstab(dataset_w_pred_anon.Ideologicaldirection, dataset_w_pred_anon.diversity_ideologicaldirection_recode, margins=True, dropna=False)

diversity_ideologicaldirection_recode,0,1,2,4,All
Ideologicaldirection,,,,,
0,473,83,37,196,789
1,230,1364,190,450,2234
2,11,4,14,20,49
3,48,42,26,83,199
4,29,54,25,483,591
All,791,1547,292,1232,3862


In [139]:
#evaluate performance against manual coding as if Llama3.1 acted as a human coder:

llm_human_column_pairs = [
( 'Acknowledgement'	,	 'interactivity_acknowledgement_updated_dum'	),
( 'TopicRelevance'	,	 'rationality_topic_relevance_dum'	),
( 'Reasoning'	,	 'rationality_reasoning_edit_dum'	),
( 'BackgroundInfo'	,	 'rationality_background_info_edit_dum'	),
( 'ExternalEvidence'	,	 'rationality_external_evidence_edit_dum'	),
( 'Opinion'	,	 'diversity_position_dum'	),
( 'disagreement'	,	 'diversity_disagreement_dum'	),
( 'Ideologicaldirection'	,	 'diversity_ideologicaldirection_recode'	),
( 'Name_calling'	,	 'incivility_namecalling_edit_dum'	),
( 'Vulgarity'	,	 'incivility_vulgarity_dum'	),
( 'Attack_reputation'	,	 'incivility_attackreputation_dum'	),
( 'Question_Intelligenc'	,	 'incivility_question_intelligence_dum'	),
( 'All_caps_function'	,	'incivility_allcaps_dum'	),
( 'Sarcasm_to_criticize'	,	 'incivility_sarcasm_dum'	),
( 'Individual_right'	,	 'intolerance_rights_dum'	),
( 'discrimination'	,	 'intolerance_discrimination_dum'	),
( 'Invoke_violence'	,	 'intolerance_violence_dum'	)
]

# Create a list to store the results
results = []

print("Inter-coder reliability: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm)) / len(np.array(human)) * 100

    # Calculate Krippendorff's alpha
    kripp_alpha = krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")

    # Append the results to the list
    results.append({
        'variable': human_col,
        'krippendorf': f"{kripp_alpha:.2f}",
        'percent_agreement': f"{agreement:.2f}%"
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Display the DataFrame as a table
print(results_df)

Inter-coder reliability: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
                variable krippendorf percent_agreement
0        Acknowledgement        0.27            68.33%
1         TopicRelevance        0.61            82.06%
2              Reasoning       -0.02            58.31%
3         BackgroundInfo        0.34            80.66%
4       ExternalEvidence        0.06            79.10%
5                Opinion        0.44            72.81%
6           disagreement       -0.06            61.76%
7   Ideologicaldirection        0.40            60.44%
8           Name_calling        0.14            65.77%
9              Vulgarity        0.37            87.26%
10     Attack_reputation        0.01            50.03%
11  Question_Intelligenc        0.01            62.33%
12     All_caps_function        0.18            74.37%
13  Sarcasm_to_criticize       -0.21            41.66%
14      Individual_right        0.02            85.34%
15        disc

In [100]:
import csv
import numpy as np
from sklearn.metrics import classification_report, cohen_kappa_score
import krippendorff

# Assuming llm_human_column_pairs and dataset_w_pred_anon are already defined

# Define the CSV file path
csv_file_path = 'inter_coder_reliability.csv'

# Create a list to store the results
results = []

# Calculate inter-coder reliability metrics
for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    classification_rep = classification_report(human, llm, output_dict=True)
    cohen_kappa = cohen_kappa_score(human, llm)
    krippendorf_alpha = krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")
    
    # Flatten the classification report dictionary
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                results.append({
                    'human_col': human_col,
                    'llm_col': llm_col,
                    'label': label,
                    'metric': metric,
                    'value': value,
                    'cohen_kappa_score': cohen_kappa,
                    'krippendorf_alpha': krippendorf_alpha
                })
        else:
            results.append({
                'human_col': human_col,
                'llm_col': llm_col,
                'label': 'overall',
                'metric': label,
                'value': metrics,
                'cohen_kappa_score': cohen_kappa,
                'krippendorf_alpha': krippendorf_alpha
            })

# Write the results to a CSV file
with open(csv_file_path, mode='w', newline='') as csv_file:
    fieldnames = ['human_col', 'llm_col', 'label', 'metric', 'value', 'cohen_kappa_score', 'krippendorf_alpha']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Results have been written to {csv_file_path}")

Results have been written to inter_coder_reliability.csv


In [34]:
#check whether lower performance of diversity_conservative__dum is due to token restrictions in options:
options_str = """
seed: 42
temperature: 0.1
num_predict: 300
"""
#note default = num_predict: 128

options_long = yaml.safe_load(options_str)

In [39]:
pubspherepromptsrun17 = ['diversity_ideological_direction']   
predictions17: typing.Dict[str, np.ndarray] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["commentText"], model=MODEL, options=options_long)
    )
    for label, path in CFG.prompt_classify_files.items() if label in pubspherepromptsrun17
}

classifying diversity_ideologicaldirection: 100%|██████████| 3862/3862 [29:22<00:00,  2.19it/s]  


In [40]:
predictions17

{'diversity_ideological_direction': 0                              Absent
 1             Left/Liberal/Democratic
 2             Left/Liberal/Democratic
 3                             Neutral
 4                              Absent
                     ...              
 3857          Left/Liberal/Democratic
 3858    Right/Conservative/Republican
 3859                           Absent
 3860                           Absent
 3861                          Neutral
 Name: diversity_ideologicaldirection, Length: 3862, dtype: object}

In [76]:
dataset_w_pred_2.columns.to_list()

['StartDate',
 'RecordedDate',
 'IPAddress',
 'Finished',
 'Coder',
 'ID',
 'Mark_ID',
 'Genre',
 'topiccode',
 'Platform',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'TopicRelevance',
 'Reasoning',
 'BackgroundInfo',
 'ExternalEvidence',
 'ExternalEvidence_1_TEXT',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'INTERACTIVITY_DUMMY',
 'RATIONALITY_DUMMY',
 'HAS_OPINION_DUMMY',
 'LIBERAL_NEUTRAL_CONSERVATIVE',
 'LIBERAL_DUMMY',
 'CONSERVATIVE_DUMMY',
 'NAMECALLING_DUMMY',
 'VULGAR_DUMMY',
 'NAMECALLING_VULGAR_DUMMY',
 'INCIVILITY_ORDINAL',
 'INCIVILITY_DUMMY',
 'INTOLERANCE_DUMMY',
 'filter_$',
 'IMPOLITENESS_DUMMY',
 'showName',
 'genre',
 'Time_comment',
 'likeCount_comment',
 'entities',
 'place',
 'retweet_count',
 'platform',
 'retweeted',
 '

In [75]:
#join to the dataset:   
for _, preds in predictions17.items():
    print(preds.value_counts())
    print("-" * 42)
    preds_df = pd.DataFrame(preds).add_suffix('_long')
   
    dataset_w_pred_2 = dataset_w_pred_2.join(preds_df)
    dataset_w_pred_anon = dataset_w_pred_anon.join(preds_df)

diversity_ideologicaldirection
Absent                           1454
Right/Conservative/Republican    1165
Left/Liberal/Democratic           805
Neutral                           320
Unclear which direction           113
Name: count, dtype: int64
------------------------------------------


In [121]:
dataset_w_pred_2.to_json(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_2.to_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.parquet')
dataset_w_pred_anon.to_json('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_anon.to_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [123]:
#compare run-rerun results:
print(pd.crosstab(dataset_w_pred_anon['diversity_ideologicaldirection'], dataset_w_pred_anon['diversity_ideologicaldirection_long'], margins=True, dropna=False))

diversity_ideologicaldirection_long  Absent  Left/Liberal/Democratic  Neutral  \
diversity_ideologicaldirection                                                  
Absent                                 1399                        6       10   
Left/Liberal/Democratic                   5                      759        9   
Neutral                                  10                       12      255   
Right/Conservative/Republican            26                       28       44   
Unclear which direction                  14                        0        1   
NaN                                       0                        0        1   
All                                    1454                      805      320   

diversity_ideologicaldirection_long  Right/Conservative/Republican  \
diversity_ideologicaldirection                                       
Absent                                                           7   
Left/Liberal/Democratic                                     

In [78]:
#recode into dummies:
dataset_w_pred_anon.loc[:, 'diversity_conservative_long_dum'] = dataset_w_pred_anon.loc[:, 'diversity_ideologicaldirection_long'].map({"Left/Liberal/Democratic": 0, "Absent":0, "Neutral":0, "Unclear which direction":0 , "Right/Conservative/Republican":1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'diversity_liberal_long_dum'] = dataset_w_pred_anon.loc[:, 'diversity_ideologicaldirection_long'].map({"Left/Liberal/Democratic": 1, "Absent":0, "Neutral":0, "Unclear which direction":0 , "Right/Conservative/Republican":0}).fillna(0).astype(int)


In [79]:
#evaluete respective performance:
llm_human_column_pairs = [
    ("LIBERAL_DUMMY", "diversity_liberal_long_dum"),
    ("CONSERVATIVE_DUMMY", "diversity_conservative_long_dum"),
    ("LIBERAL_DUMMY", "diversity_liberal_dum"),
    ("CONSERVATIVE_DUMMY", "diversity_conservative_dum")
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      3073
           1       0.59      0.60      0.60       789

    accuracy                           0.83      3862
   macro avg       0.74      0.75      0.75      3862
weighted avg       0.83      0.83      0.83      3862

cohen_kappa_score(LIBERAL_DUMMY): 0.4909411843205673
krippendorf(LIBERAL_DUMMY): 0.490993755129861
---
              precision    recall  f1-score   support

           0       0.96      0.79      0.86      3271
           1       0.41      0.80      0.54       591

    accuracy                           0.79      3862
   macro avg       0.68      0.79      0.70      3862
weighted avg       0.87      0.79      0.81      3862

cohen_kappa_score(CONSERVATIVE_DUMMY): 0.42119755573079587
krippendorf(CONSERVATIVE_DUMMY): 0.40307780447886066
---
              prec

very similar performance regardless of num_predict token limit settings, makes sense since this limit is about max output tokens in the response, not the prompt, so here is also another test-retest result for the same prompt

In [80]:
dataset_w_pred_2.to_json(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_2.to_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.parquet')
dataset_w_pred_anon.to_json('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_anon.to_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [36]:
#what if we combine rationality items into one prompt:
pubspherepromptsrun18 = ['rationality_combine']   
predictions18: typing.Dict[str, np.ndarray] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["commentText"], model=MODEL, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label in pubspherepromptsrun18
}

classifying rationality_combine: 100%|██████████| 3862/3862 [1:30:14<00:00,  1.40s/it]


In [43]:
#what if we combine incivility items into one prompt:
pubspherepromptsrun20 = ['incivility_simple', 'rationality_simple']   
predictions20: typing.Dict[str, np.ndarray] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["commentText"], model=MODEL, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label in pubspherepromptsrun20
}

classifying rationality_combine: 100%|██████████| 3862/3862 [33:51<00:00,  1.90it/s]


In [54]:
#what if we combine incivility items into one prompt:
pubspherepromptsrun21 = ['interactivity_acknowledgement_simple', 'incivility_simple2', 'rationality_simple2']   
predictions21: typing.Dict[str, np.ndarray] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["commentText"], model=MODEL, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label in pubspherepromptsrun21
}

classifying rationality_simple2: 100%|██████████| 3862/3862 [46:36<00:00,  1.38it/s] 


In [37]:
#join to the dataset:   
for _, preds in predictions18.items():
    print(preds.value_counts())
    print("-" * 42)
     
    dataset_w_pred_2 = dataset_w_pred_2.join(preds, rsuffix='_exactexample')
    dataset_w_pred_anon = dataset_w_pred_anon.join(preds, rsuffix='_exactexample')

rationality_combine
No     2473
Yes    1386
Name: count, dtype: int64
------------------------------------------


In [45]:
dataset_w_pred_anon.columns.to_list()

['StartDate',
 'RecordedDate',
 'IPAddress',
 'Finished',
 'Coder',
 'ID',
 'Mark_ID',
 'Genre',
 'topiccode',
 'Platform',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'TopicRelevance',
 'Reasoning',
 'BackgroundInfo',
 'ExternalEvidence',
 'ExternalEvidence_1_TEXT',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'INTERACTIVITY_DUMMY',
 'RATIONALITY_DUMMY',
 'HAS_OPINION_DUMMY',
 'LIBERAL_NEUTRAL_CONSERVATIVE',
 'LIBERAL_DUMMY',
 'CONSERVATIVE_DUMMY',
 'NAMECALLING_DUMMY',
 'VULGAR_DUMMY',
 'NAMECALLING_VULGAR_DUMMY',
 'INCIVILITY_ORDINAL',
 'INCIVILITY_DUMMY',
 'INTOLERANCE_DUMMY',
 'filter_$',
 'IMPOLITENESS_DUMMY',
 'showName',
 'genre',
 'Time_comment',
 'likeCount_comment',
 'entities',
 'place',
 'retweet_count',
 'platform',
 'retweeted',
 '

In [39]:
#compare influence of exact versus similar example:
print(pd.crosstab(dataset_w_pred_anon['rationality_combine_exactexample'], dataset_w_pred_anon['rationality_combine'], margins=True, dropna=False))

rationality_combine                 No   Yes  NaN     All
rationality_combine_exactexample                         
No                                2394    79    0  2473.0
Yes                                 48  1338    0  1386.0
NaN                                  1     0    2     NaN
All                               2443  1417    0  3862.0


very small effect

In [55]:
#join to the dataset:   
for _, preds in predictions21.items():
    print(preds.value_counts())
    print("-" * 42)
     
    dataset_w_pred_2 = dataset_w_pred_2.join(preds, rsuffix='_2')
    dataset_w_pred_anon = dataset_w_pred_anon.join(preds, rsuffix='_2')

incivility_simple2
Yes    2414
No     1448
Name: count, dtype: int64
------------------------------------------
interactivity_acknowledgement_simple
No     2318
Yes    1544
Name: count, dtype: int64
------------------------------------------
rationality_simple2
No     3489
Yes     373
Name: count, dtype: int64
------------------------------------------


In [56]:
dataset_w_pred_anon.loc[:, 'rationality_combine_exactexample_dum'] = dataset_w_pred_2.loc[:, 'rationality_combine_exactexample'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_combine_dum'] = dataset_w_pred_2.loc[:, 'rationality_combine'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_simple_dum'] = dataset_w_pred_2.loc[:, 'rationality_simple'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_combine_dum'] = dataset_w_pred_2.loc[:, 'incivility_combined'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_simple_dum'] = dataset_w_pred_2.loc[:, 'incivility_simple'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_simple2_dum'] = dataset_w_pred_2.loc[:, 'incivility_simple2'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_simple2_dum'] = dataset_w_pred_2.loc[:, 'rationality_simple2'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_simple_dum'] = dataset_w_pred_2.loc[:, 'interactivity_acknowledgement_simple'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

In [57]:
dataset_w_pred_2.to_json(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_2.to_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.parquet')
dataset_w_pred_anon.to_json('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_anon.to_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [58]:
#evaluete respective performance (NOTE the wording of the few shot examples in the prompt is polished by GPT4o, not exactly like codebook):
llm_human_column_pairs = [
    ("RATIONALITY_DUMMY", "rationality_combine_dum"),
    ("RATIONALITY_DUMMY", "rationality_combine_exactexample_dum"),
    ("RATIONALITY_DUMMY", "rationality_simple_dum"),
    ("RATIONALITY_DUMMY", "rationality_simple2_dum"),
    ("INCIVILITY_DUMMY", "incivility_combine_dum"),
    ("INCIVILITY_DUMMY", "incivility_simple_dum"),
    ("INCIVILITY_DUMMY", "incivility_simple2_dum"),
    ("INTERACTIVITY_DUMMY", "interactivity_acknowledgement_simple_dum")
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.93      0.72      0.81      3165
           1       0.38      0.77      0.51       697

    accuracy                           0.73      3862
   macro avg       0.66      0.74      0.66      3862
weighted avg       0.83      0.73      0.76      3862

cohen_kappa_score(RATIONALITY_DUMMY): 0.34728201946365866
krippendorf(RATIONALITY_DUMMY): 0.31883884197869394
percent_agreement(RATIONALITY_DUMMY): 72.92%
---
              precision    recall  f1-score   support

           0       0.93      0.73      0.82      3165
           1       0.38      0.76      0.51       697

    accuracy                           0.74      3862
   macro avg       0.66      0.75      0.67      3862
weighted avg       0.83      0.74      0.76      3862

cohen_kappa_score(RATIONALITY_DUMMY): 0.35617137578933
krippendorf(RATIONALITY

In [20]:
#These are the legacy results of the old prompts:

print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.94      0.85      0.89      3073
           1       0.56      0.77      0.65       789

    accuracy                           0.83      3862
   macro avg       0.75      0.81      0.77      3862
weighted avg       0.86      0.83      0.84      3862

cohen_kappa_score(LIBERAL_DUMMY): 0.5444043080556233
krippendorf(LIBERAL_DUMMY): 0.5408921571554941
---
              precision    recall  f1-score   support

           0       0.95      0.90      0.92      3271
           1       0.57      0.74      0.65       591

    accuracy                           0.88      3862
   macro avg       0.76      0.82      0.79      3862
weighted avg       0.89      0.88      0.88      3862

cohen_kappa_score(CONSERVATIVE_DUMMY): 0.5725715490094205
krippendorf(CONSERVATIVE_DUMMY): 0.571045663399131
---
              precis

In [4]:
MODELS: typing.Tuple[str, str] = [
    ("topics", "cardiffnlp/tweet-topic-21-multi"),
    ("emotions", "cardiffnlp/twitter-roberta-base-emotion-multilabel-latest"),
    ("sentiment", "cardiffnlp/twitter-roberta-base-sentiment-latest"),
    ("irony", "cardiffnlp/twitter-roberta-base-irony"),
    ("offensive",  "cardiffnlp/twitter-roberta-base-offensive"),
    ("hate", "cardiffnlp/twitter-roberta-base-hate-latest")
]

In [5]:
Cardiff_predictions: typing.Dict[str, pd.Series] = {}

In [6]:
for label, model in MODELS:
    classifier = src.HFClassify(slug=model)
    Cardiff_predictions[label] = pd.Series(classifier(dataset["commentText"], theta=0.4))

100%|██████████| 121/121 [07:06<00:00,  3.52s/it]
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 121/121 [07:04<00:00,  3.50s/it]


In [8]:
Cardiff_predictions

{'topics': 0                                                      {}
 1                                 {news_&_social_concern}
 2                                 {news_&_social_concern}
 3                                 {news_&_social_concern}
 4                                  {diaries_&_daily_life}
                               ...                        
 3857    {news_&_social_concern, business_&_entrepreneu...
 3858                              {news_&_social_concern}
 3859             {film_tv_&_video, news_&_social_concern}
 3860                              {news_&_social_concern}
 3861                              {news_&_social_concern}
 Length: 3862, dtype: object,
 'emotions': 0                  {pessimism, sadness}
 1                      {disgust, anger}
 2        {anticipation, anger, disgust}
 3                             {disgust}
 4                             {sadness}
                      ...               
 3857                   {disgust, anger}
 3858       

In [9]:
for label, series in Cardiff_predictions.items():
    display(label)
    display(series.value_counts())

'topics'

{news_&_social_concern}                                                                        2161
{diaries_&_daily_life}                                                                          240
{}                                                                                              208
{film_tv_&_video, news_&_social_concern}                                                        146
{diaries_&_daily_life, news_&_social_concern}                                                    97
                                                                                               ... 
{diaries_&_daily_life, news_&_social_concern, music}                                              1
{film_tv_&_video, arts_&_culture}                                                                 1
{news_&_social_concern, youth_&_student_life, sports, learning_&_educational}                     1
{film_tv_&_video, news_&_social_concern, business_&_entrepreneurs, celebrity_&_pop_culture}       1


'emotions'

{disgust, anger}                       1481
{disgust, anger, sadness}               278
{anticipation}                          263
{}                                      210
{joy, optimism}                         200
                                       ... 
{optimism, fear, sadness}                 1
{anticipation, fear, disgust}             1
{disgust, pessimism, fear, sadness}       1
{fear, surprise}                          1
{anticipation, joy, anger}                1
Name: count, Length: 86, dtype: int64

'sentiment'

...

{non_irony, irony}    1874
{non_irony}           1042
{irony}                946
Name: count, dtype: int64

'offensive'

{non-offensive}               2066
{offensive, non-offensive}    1158
{offensive}                    638
Name: count, dtype: int64

'hate'

{NOT-HATE}          3532
{HATE}               276
{HATE, NOT-HATE}      54
Name: count, dtype: int64

In [ ]:
dataset_w_pred = pd.read_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify.parquet')

for labels, preds in Cardiff_predictions.items():
    dataset_w_pred_2 = dataset_w_pred.join(preds.rename(labels))

dataset_w_pred_2

In [11]:
for labels, preds in Cardiff_predictions.items():
    dataset_w_pred_2 = dataset_w_pred_2.join(preds.rename(labels))

dataset_w_pred_2

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,rationality_topic_relevance,political_negativity,rationality_background_info,rationality_reasoning,topics,emotions,sentiment,irony,offensive,hate
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,No,not political/negative,No,No,{},"{pessimism, sadness}","{neutral, negative}","{non_irony, irony}","{offensive, non-offensive}",{NOT-HATE}
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,Yes,not political/negative,No,Yes,{news_&_social_concern},"{disgust, anger}","{neutral, negative}",{non_irony},"{offensive, non-offensive}",{NOT-HATE}
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,Yes,political/negative,No,No,{news_&_social_concern},"{anticipation, anger, disgust}","{neutral, negative}",{non_irony},{non-offensive},{NOT-HATE}
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,Yes,political/negative,Yes,Yes,{news_&_social_concern},{disgust},"{neutral, negative}",{irony},{non-offensive},{NOT-HATE}
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,No,not political/negative,No,No,{diaries_&_daily_life},{sadness},"{neutral, positive}","{non_irony, irony}",{non-offensive},{NOT-HATE}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,8/19/2021 14:50:13,8/19/2021 14:54:28,62.194.51.29,1,6,1152219467579100000,10000695,0,4,2,...,Yes,political/negative,No,Yes,"{news_&_social_concern, business_&_entrepreneu...","{disgust, anger}","{neutral, negative}",{non_irony},{non-offensive},{HATE}
3858,8/19/2021 15:10:27,8/19/2021 15:12:21,62.194.51.29,1,6,1085362296472430000,10007008,1,4,2,...,Yes,political/negative,No,No,{news_&_social_concern},{anticipation},{neutral},{non_irony},{non-offensive},{NOT-HATE}
3859,10/6/2021 16:08:39,10/6/2021 16:10:42,213.127.113.113,1,6,UghFY3QJ6nmT_ngCoAEC.7-H0Z7--wxd8goqpaPs-bl,20000102,0,3,1,...,No,not political/negative,No,Yes,"{film_tv_&_video, news_&_social_concern}",{anticipation},"{neutral, negative}",{irony},{non-offensive},{NOT-HATE}
3860,10/15/2021 18:30:04,10/15/2021 18:35:40,213.127.109.191,1,6,UgyWabsmmnq3zam4DgZ4AaABAg,20000418,2,3,1,...,No,not political/negative,No,No,{news_&_social_concern},"{anticipation, optimism, trust}","{neutral, positive}",{non_irony},{non-offensive},{NOT-HATE}


In [14]:
# Select all columns that start with 'cardiff'
cardiff_columns = ['sentiment', 'offensive', 'topics', 'emotions', 'irony', 'hate']

# Convert each selected column to categorical type
for col in cardiff_columns:
    dataset_w_pred_2[col] = dataset_w_pred_2[col].astype('str').astype('category')

# Display the DataFrame to verify the changes
print(dataset_w_pred_2)

                StartDate         RecordedDate        IPAddress  Finished  \
0      5/30/2021 13:03:17   5/30/2021 13:04:17     62.194.51.29         1   
1     10/11/2021 10:34:05  10/11/2021 10:36:46  213.127.109.191         1   
2       9/9/2021 18:49:48    9/9/2021 18:51:32    213.127.110.0         1   
3       6/6/2021 16:12:46    6/6/2021 16:16:16   213.127.76.145         1   
4      6/13/2021 13:25:49   6/13/2021 13:27:28   213.127.82.232         1   
...                   ...                  ...              ...       ...   
3857   8/19/2021 14:50:13   8/19/2021 14:54:28     62.194.51.29         1   
3858   8/19/2021 15:10:27   8/19/2021 15:12:21     62.194.51.29         1   
3859   10/6/2021 16:08:39   10/6/2021 16:10:42  213.127.113.113         1   
3860  10/15/2021 18:30:04  10/15/2021 18:35:40  213.127.109.191         1   
3861  11/19/2021 17:49:17  11/19/2021 17:51:04  213.127.109.191         1   

      Coder                                                 ID   Mark_ID  \

In [18]:
dataset_w_pred_2.to_json(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_2.to_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.parquet')

In [16]:
#anonymize:
dataset_w_pred_anon = dataset_w_pred_2.drop('commentText', axis=1)

In [17]:
dataset_w_pred_anon.to_json('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_anon.to_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [75]:
dataset_w_pred_anon = pd.read_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [12]:
dataset_w_pred_anon.loc[:, 'hate'].value_counts()

hate
{'NOT-HATE'}            3532
{'HATE'}                 276
{'HATE', 'NOT-HATE'}      54
Name: count, dtype: int64

In [76]:
#recode offensive to a dummy with non-offensive as 0 and offensive as 1:
dataset_w_pred_anon.loc[:, 'offensive_dum'] = dataset_w_pred_anon.loc[:, 'offensive'].map({"{'non-offensive'}": 0, "{'offensive', 'non-offensive'}": 1, "{'offensive'}": 1})
#recode hate into a dummy with non-hate as 0 and hate as 1:
dataset_w_pred_anon.loc[:, 'hate_dum'] = dataset_w_pred_anon.loc[:, 'hate'].map({"{'NOT-HATE'}": 0, "{'HATE'}": 1, "{'HATE', 'NOT-HATE'}": 1})

In [77]:
dataset_w_pred_anon.loc[:, 'cardiff_incivil'] = dataset_w_pred_anon.loc[:, ['offensive_dum', 'hate_dum']].max(axis=1)

In [78]:
print(dataset_w_pred_anon.loc[:, 'offensive'].value_counts())
print(dataset_w_pred_anon.loc[:, 'offensive_dum'].value_counts())
print(dataset_w_pred_anon.loc[:, 'hate'].value_counts())
print(dataset_w_pred_anon.loc[:, 'hate_dum'].value_counts())

offensive
{'non-offensive'}                 2066
{'offensive', 'non-offensive'}    1158
{'offensive'}                      638
Name: count, dtype: int64
offensive_dum
0    2066
1    1796
Name: count, dtype: int64
hate
{'NOT-HATE'}            3532
{'HATE'}                 276
{'HATE', 'NOT-HATE'}      54
Name: count, dtype: int64
hate_dum
0    3532
1     330
Name: count, dtype: int64


In [16]:
dimensions = pd.DataFrame([['Incivility',
               ['INCIVILITY_DUMMY','HATELIST_FOCUSED_DUMMY'],
               ['cardiff_incivil','hate_dum','offensive_dum'	
               ]]]  , 
            columns = ['dimension','labels','measures'])
dimensions

,dimension,labels,measures
0,Incivility,"[INCIVILITY_DUMMY, HATELIST_FOCUSED_DUMMY]","[cardiff_incivil, hate_dum, offensive_dum]"


In [17]:
print(dataset_w_pred_anon.hate_dum.dtypes)  # Verify the data types

int64


In [20]:
#calculate performance of cardiff's incivility classification:

# Reinitialize the DataFrame
performance = pd.DataFrame(columns=['Dimension', 'Label', 'Measures', 'Class', 'Precision', 'Recall', 'F1-score', 'support', 'Accuracy_overall'])

classif = ['0', '1', 'macro avg', 'weighted avg']

for index, dim in dimensions.iterrows():
    for label in dim['labels']:
        for measure in dim['measures']:
            try:
                data = dataset_w_pred_anon[dataset_w_pred_anon[measure].notna()]
                classfication = classification_report(data[label], data[measure], output_dict=True)
                for c in classif:
                    row = {
                        'Dimension': dim['dimension'],
                        'Label': label,
                        'Measures': measure,
                        'Class': c,
                        'Precision': classfication[c]['precision'],
                        'Recall': classfication[c]['recall'],
                        'F1-score': classfication[c]['f1-score'],
                        'support': classfication[c]['support'],
                        'Accuracy_overall': classfication['accuracy']
                    }
                    performance = pd.concat([performance, pd.DataFrame([row])], ignore_index=True)
            except IndexError:
                pass

C:\Users\sstolwi\AppData\Local\Temp\ipykernel_11660\3664728362.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  performance = pd.concat([performance, pd.DataFrame([row])], ignore_index=True)


In [21]:
performance

,Dimension,Label,Measures,Class,Precision,Recall,F1-score,support,Accuracy_overall
0,Incivility,INCIVILITY_DUMMY,cardiff_incivil,0,0.725886,0.746835,0.736212,1975.0,0.726308
1,Incivility,INCIVILITY_DUMMY,cardiff_incivil,1,0.726776,0.704822,0.715631,1887.0,0.726308
2,Incivility,INCIVILITY_DUMMY,cardiff_incivil,macro avg,0.726331,0.725829,0.725921,3862.0,0.726308
3,Incivility,INCIVILITY_DUMMY,cardiff_incivil,weighted avg,0.726321,0.726308,0.726156,3862.0,0.726308
4,Incivility,INCIVILITY_DUMMY,hate_dum,0,0.544734,0.974177,0.698747,1975.0,0.570430
5,Incivility,INCIVILITY_DUMMY,hate_dum,1,0.845455,0.147854,0.251691,1887.0,0.570430
6,Incivility,INCIVILITY_DUMMY,hate_dum,macro avg,0.695094,0.561015,0.475219,3862.0,0.570430
7,Incivility,INCIVILITY_DUMMY,hate_dum,weighted avg,0.691668,0.570430,0.480313,3862.0,0.570430
8,Incivility,INCIVILITY_DUMMY,offensive_dum,0,0.724105,0.757468,0.740411,1975.0,0.728379
9,Incivility,INCIVILITY_DUMMY,offensive_dum,1,0.733296,0.697933,0.715178,1887.0,0.728379


In [ ]:
#pretty good for incivility and on-par with ML model based on bert-base-uncased, fine-tuned on this dataset

In [45]:
dataset_w_pred_2.to_json(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_2.to_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.parquet')

In [46]:
#anonymize:
dataset_w_pred_anon = dataset_w_pred_2.drop('commentText', axis=1)
dataset_w_pred_anon.to_json('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_anon.to_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [53]:
dimensions = pd.DataFrame([['Diversity',
               ['HAS_OPINION_DUMMY','LIBERAL_DUMMY','CONSERVATIVE_DUMMY'],
               ['political_conservative_US', 'political_liberal_US', 'political_opinion_US'
               ]]]  , 
            columns = ['dimension','labels','measures'])
dimensions

,dimension,labels,measures
0,Diversity,"[HAS_OPINION_DUMMY, LIBERAL_DUMMY, CONSERVATIV...","[political_conservative_US, political_liberal_..."


In [54]:
#calculate performance of cardiff's incivility classification:

# Reinitialize the DataFrame
performance = pd.DataFrame(columns=['Dimension', 'Label', 'Measures', 'Class', 'Precision', 'Recall', 'F1-score', 'support', 'Accuracy_overall'])

classif = ['0', '1', 'macro avg', 'weighted avg']

for index, dim in dimensions.iterrows():
    for label in dim['labels']:
        for measure in dim['measures']:
            try:
                data = dataset_w_pred_anon[dataset_w_pred_anon[measure].notna()]
                classfication = classification_report(data[label], data[measure], output_dict=True)
                for c in classif:
                    row = {
                        'Dimension': dim['dimension'],
                        'Label': label,
                        'Measures': measure,
                        'Class': c,
                        'Precision': classfication[c]['precision'],
                        'Recall': classfication[c]['recall'],
                        'F1-score': classfication[c]['f1-score'],
                        'support': classfication[c]['support'],
                        'Accuracy_overall': classfication['accuracy']
                    }
                    performance = pd.concat([performance, pd.DataFrame([row])], ignore_index=True)
            except IndexError:
                pass

C:\Users\sstolwi\AppData\Local\Temp\ipykernel_4848\3664728362.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  performance = pd.concat([performance, pd.DataFrame([row])], ignore_index=True)


In [55]:
performance  #these are legacy variables, did not set seed or temperature, so results are not reproducible

,Dimension,Label,Measures,Class,Precision,Recall,F1-score,support,Accuracy_overall
0,Diversity,HAS_OPINION_DUMMY,political_conservative_US,0,0.572076,0.926062,0.707249,1907.0,0.621440
1,Diversity,HAS_OPINION_DUMMY,political_conservative_US,1,0.818065,0.324297,0.464469,1955.0,0.621440
2,Diversity,HAS_OPINION_DUMMY,political_conservative_US,macro avg,0.695070,0.625179,0.585859,3862.0,0.621440
3,Diversity,HAS_OPINION_DUMMY,political_conservative_US,weighted avg,0.696599,0.621440,0.584350,3862.0,0.621440
4,Diversity,HAS_OPINION_DUMMY,political_liberal_US,0,0.602440,0.880440,0.715381,1907.0,0.654065
5,Diversity,HAS_OPINION_DUMMY,political_liberal_US,1,0.787907,0.433248,0.559076,1955.0,0.654065
6,Diversity,HAS_OPINION_DUMMY,political_liberal_US,macro avg,0.695173,0.656844,0.637229,3862.0,0.654065
7,Diversity,HAS_OPINION_DUMMY,political_liberal_US,weighted avg,0.696326,0.654065,0.636257,3862.0,0.654065
8,Diversity,HAS_OPINION_DUMMY,political_opinion_US,0,0.764414,0.806502,0.784894,1907.0,0.781719
9,Diversity,HAS_OPINION_DUMMY,political_opinion_US,1,0.800541,0.757545,0.778449,1955.0,0.781719


In [ ]:
#very good performance for all political ideology classifications


In [103]:
#these are legacy variables, did not set seed or temperature, so results are not reproducible:
llm_human_column_pairs = [
    ("LIBERAL_DUMMY", "political_liberal_US"),
    ("CONSERVATIVE_DUMMY", "political_conservative_US"),
    ("HAS_OPINION_DUMMY", "political_opinion_US"),
    ("RATIONALITY_DUMMY", "rationality_prompt_dum"),
   # ("TopicRelevance", "rationality_topic_relevance"),
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon[[human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()
    
    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.93      0.85      0.89      3073
           1       0.56      0.77      0.65       789

    accuracy                           0.83      3862
   macro avg       0.75      0.81      0.77      3862
weighted avg       0.86      0.83      0.84      3862

cohen_kappa_score(LIBERAL_DUMMY): 0.5409748950370545
krippendorf(LIBERAL_DUMMY): 0.5375974087799733
---
              precision    recall  f1-score   support

           0       0.95      0.90      0.92      3271
           1       0.57      0.74      0.64       591

    accuracy                           0.87      3862
   macro avg       0.76      0.82      0.78      3862
weighted avg       0.89      0.87      0.88      3862

cohen_kappa_score(CONSERVATIVE_DUMMY): 0.5676841210411792
krippendorf(CONSERVATIVE_DUMMY): 0.566055054744786
---
              precis

In [ ]:
#although F1-score for rationality is decent, the cohen's kappa is very low, indicating poor agreement between human and LLM classification, probably due to low precision on positive class
#performance is on-par than supervised machine learning models in the public sphere paper, but less worse than finetuned bert-base-uncased model (0.72 macro avg precision, recall and f1)
#however krippendorf of manual coding for rationality items was in same range as LLM classification, indicating that the task is difficult to classify
#apparently mainly the reasoning/argumentation item is difficult to classify (manual K-alpha = 0.3; llm-human K-alpha = 0.05), as the other two items have a higher agreement between human and LLM classification

In [104]:
dataset_w_pred_anon.to_json('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_anon.to_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [21]:
dataset_w_pred_anon = pd.read_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [2]:
#setup a dummy dataset for incivility to provide to Simon to build a classifyer:
#read in the data:
dataset_w_pred_2 = pd.read_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.parquet')

In [8]:
# Replace the values in the commentText column with sequential text
pubsphere_incivility_dummy = dataset_w_pred_2.loc[:5, ["commentText", "INCIVILITY_DUMMY"]].head()
for idx in range(len(pubsphere_incivility_dummy)):
    pubsphere_incivility_dummy.at[idx, 'commentText'] = f"This is an English language post {idx + 1}"

# Display the DataFrame to verify the changes
print(pubsphere_incivility_dummy)

                          commentText  INCIVILITY_DUMMY
0  This is an English language post 1                 0
1  This is an English language post 2                 1
2  This is an English language post 3                 0
3  This is an English language post 4                 0
4  This is an English language post 5                 0


In [11]:
#save data:
pubsphere_incivility_dummy.to_parquet('data/publicsphere/publicsphere_incivility_dummy.parquet')